In [1]:
import pandas as pd
import nltk

#nltk.download('stopwords')
#nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\giuli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\giuli\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# ANGER_LEXICO = "./lexico/anger-NRC-Emotion-Lexicon.txt"
# ANTICIPATION_LEXICO = "./lexico/anticipation-NRC-Emotion-Lexicon.txt"
# DISGUST_LEXICO = "./lexico/disgust-NRC-Emotion-Lexicon.txt"
# FEAR_LEXICO = "./lexico/fear-NRC-Emotion-Lexicon.txt"
# JOY_LEXICO = "./lexico/joy-NRC-Emotion-Lexicon.txt"
# NEGATIVE_LEXICO = "./lexico/negative-NRC-Emotion-Lexicon.txt" # TODO; remove?
# POSITIVE_LEXICO = "./lexico/positive-NRC-Emotion-Lexicon.txt"
# SADNESS_LEXICO = "./lexico/sadness-NRC-Emotion-Lexicon.txt"
# SUPRISE_LEXICO = "./lexico/surprise-NRC-Emotion-Lexicon.txt"
# TRUST_LEXICO = "./lexico/trust-NRC-Emotion-Lexicon.txt" # TODO; remove?

lexico_categories = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'hope']


In [12]:
category_lexico_dfs = {}
for category in lexico_categories:
	category_lexico_dfs[category] = pd.read_csv(f"./lexico/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)

C:\Users\giuli\AppData\Local\Temp\ipykernel_60752\2959296284.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  category_lexico_dfs[category] = pd.read_csv(f"./lexico/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_60752\2959296284.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  category_lexico_dfs[category] = pd.read_csv(f"./lexico/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_60752\2959296284.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can av

In [28]:
if 'ham' in category_lexico_dfs['disgust']['word'].tolist():
    print('yes')
    category_lexico_dfs['disgust'][category_lexico_dfs['disgust']['word'] == 'baptist']

yes


In [13]:
lexico = pd.DataFrame(columns=['word'])
for category in lexico_categories:
    lexico = pd.merge(lexico, category_lexico_dfs[category], on='word', how='outer')
lexico

,word,anger,disgust,fear,joy,sadness,surprise,hope
0,idiotic,1,1,0,0,0,0,0
1,offend,1,1,0,0,0,0,0
2,strained,1,0,0,0,0,0,0
3,punishment,1,1,1,0,0,0,0
4,kicking,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
14149,brigade,0,0,1,0,0,0,0
14150,recurring,0,0,0,0,0,0,0
14151,pharmacist,0,0,0,0,0,0,0
14152,caress,0,0,0,0,0,0,0


In [16]:
lexico.dropna(inplace=True)

In [18]:
TRAIN_DATA_PATH = "./datasets/WASSA23_essay_level_with_labels_train.tsv"
train_df = pd.read_csv(TRAIN_DATA_PATH, sep='\t')

In [19]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def lemmatize(text):
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split())

def stemming(text):
    return ' '.join(stemmer.stem(word) for word in text.split())

lexico['preproc_word'] = ''
for index, row in lexico.iterrows():
    lexico.loc[index, 'preproc_word'] = stemming(lemmatize(row['word']))
lexico

,word,anger,disgust,fear,joy,sadness,surprise,hope,preproc_word
0,idiotic,1,1,0,0,0,0,0,idiot
1,offend,1,1,0,0,0,0,0,offend
2,strained,1,0,0,0,0,0,0,strain
3,punishment,1,1,1,0,0,0,0,punish
4,kicking,1,0,0,0,0,0,0,kick
...,...,...,...,...,...,...,...,...,...
14149,brigade,0,0,1,0,0,0,0,brigad
14150,recurring,0,0,0,0,0,0,0,recur
14151,pharmacist,0,0,0,0,0,0,0,pharmacist
14152,caress,0,0,0,0,0,0,0,caress


In [60]:
for index, row in lexico.iterrows():
    count = 0
    for category in lexico_categories: count += row[f'{category}']    
    if count == 0: lexico = lexico.drop(index=index, axis=0)
    else: 
        for index2, row2 in lexico[index+1:].iterrows():
            if row2['preproc_word'] == row['preproc_word']: # TODO: da qua non funziona
                print('si') 
                for category in lexico_categories: 
                    row[f'{category}'] += row2[f'{category}']
                lexico = lexico.drop(index=index2, axis=0)
            for category in lexico_categories: 
                if row[f'{category}'] > 1: row[f'{category}'] = 1

In [61]:
lexico

,word,anger,disgust,fear,joy,sadness,surprise,hope,preproc_word
0,idiotic,1,1,0,0,0,0,0,idiot
1,offend,1,1,0,0,0,0,0,offend
2,strained,1,0,0,0,0,0,0,strain
3,punishment,1,1,1,0,0,0,0,punish
4,kicking,1,0,0,0,0,0,0,kick
...,...,...,...,...,...,...,...,...,...
14138,radiant,0,0,0,1,0,0,0,radiant
14141,chicken,0,0,1,0,0,0,0,chicken
14143,glimmer,0,0,0,1,0,1,1,glimmer
14149,brigade,0,0,1,0,0,0,0,brigad


In [62]:
lexico.to_csv('./lexico/lexico.csv', index=False)

In [63]:
for category in lexico_categories:
	train_df[f'{category}_count'] = [0 for _ in range(len(train_df))]

for index, row in train_df.iterrows():
	preproc_essay_words = stemming(lemmatize(row['essay']))
	#print(preproc_essay_words)
	#break
	for word in preproc_essay_words.split():
		if word in lexico['preproc_word'].tolist():
			for category in lexico_categories:
				train_df.loc[index, f'{category}_count'] += lexico[lexico['preproc_word'] == word][category].tolist()[0]
				#if lexico[lexico['preproc_word'] == word][category].tolist()[0]==1:
				# 	print(word)
				# 	print(category)
				# 	print("associate")
				#train_df[index][f'{category}_count'] += lexico[lexico['preproc_word'] == word][category].tolist()[0]
	#break
train_df


,conversation_id,article_id,essay,empathy,distress,speaker_id,gender,education,race,age,...,split,essay_id,emotion,anger_count,disgust_count,fear_count,joy_count,sadness_count,surprise_count,hope_count
0,2,35,It breaks my heart to see people living in tho...,6.833333,6.625,30,1,6,3,37,...,train,1,Hope/Sadness,4,2,4,3,3,4,3
1,3,35,I wonder why there aren't more people trying t...,5.833333,6.000,19,1,6,2,32,...,train,2,Anger,1,1,1,4,1,1,0
2,5,35,"After reading the article, you can't help but ...",1.000000,1.375,17,1,6,1,29,...,train,4,Sadness,5,4,4,8,5,2,6
3,6,213,It is so sad that someone who had such an amaz...,6.166667,6.625,16,2,5,1,28,...,train,5,Sadness,4,3,4,1,6,3,1
4,8,213,"From reading the article, it looks like the wo...",6.833333,1.000,30,1,6,3,37,...,train,7,Neutral,2,1,1,4,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,495,218,I feel that this will become a national proble...,6.500000,6.750,30,1,6,3,37,...,train,994,Neutral,8,5,5,3,9,3,3
788,496,103,The whole situation is sketchy. The wavering r...,3.166667,6.375,16,2,5,1,28,...,train,995,Neutral,5,2,7,1,4,1,0
789,498,103,The death of a former aide to Russian Presiden...,6.000000,2.000,43,2,6,1,33,...,train,997,Neutral,2,2,4,1,4,2,0
790,499,103,Everything about Russia really freaks me out. ...,6.000000,6.000,53,2,3,1,27,...,train,998,Fear,5,3,8,3,2,1,3


In [22]:
train_df['essay'][0]

'It breaks my heart to see people living in those conditions. I hope that all the aid that was sent to the island makes it to the people who need it the most. I do not know what I would do it that was my family and I. I would hope that I would do my best, but I can see how depressing and hopeless you could feel having your whole life changed because of a storm and not knowing where your next meal is coming from.'

In [23]:
# check if the stirng 'hope' is in lexico['preproc_word']
if 'hope' in lexico['preproc_word'].to_list():
    print("trovato")


trovato


In [24]:
lexico['preproc_word']

0             idiot
1            offend
2            strain
3            punish
4              kick
            ...    
14149        brigad
14150         recur
14151    pharmacist
14152        caress
14153         humor
Name: preproc_word, Length: 14153, dtype: object

In [25]:
stemming(lemmatize('person'))

'person'

In [26]:
# merge category_lexico_dfs['anger'] and category_lexico_dfs['disgust'] on 'word' and rename 'associated' to 'anger' and 'disgust'